# Notebook 1: Enhancing Images with Stable Diffusion Img2Img Pipeline


### 1. Install Required Libraries

In [ ]:
# Run this cell to install necessary packages
!pip install --upgrade diffusers transformers torch torchvision ipywidgets
!pip install opencv-python matplotlib

#### 3.1 Optional: Install `accelerate` for faster and less memory-intense model loading.

In [ ]:
!pip install accelerate

### 2. Import Libraries

In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline, UniPCMultistepScheduler
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
### 3. Load Models

In [ ]:
# Load the Stable Diffusion img2img pipeline
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)

In [ ]:
# Enable GPU acceleration
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

In [ ]:
# Use an efficient scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

### 4. Prepare Input Image

In [ ]:
# Load the input image
input_image = Image.open("test-images/20240929_102048-EDIT.jpg").convert("RGB")
input_image = input_image.resize((512, 512))

plt.imshow(input_image)
plt.axis('off')  # Hide the axes
plt.show()

### 5. Define Parameters

In [ ]:
# Set the denoising strength (lower value = closer to input, higher value = more artistic freedom)
denoising_strength = 0.5

# Set the number of sampling steps
num_inference_steps = 50

prompt = ""
negative_prompt = "low quality, blurry, deformed, bad anatomy"
guidance_scale = 7.5

### 6. Generate the Enhanced Image

In [ ]:
# Generate the enhanced image
with torch.autocast(device):
    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        init_image=input_image,
        strength=denoising_strength,
    )

enhanced_image = output.images[0]

### 7. Display and Save the Results

In [ ]:
# Save the enhanced image
enhanced_image.save("output-images/enhanced_img2img.jpg")

In [ ]:
# Display the original and enhanced images
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].imshow(input_image)
ax[0].set_title("Original Image")
ax[0].axis("off")

ax[1].imshow(enhanced_image)
ax[1].set_title("Enhanced Image")
ax[1].axis("off")

plt.show()

### 8. Style Transfer
By using a higher denoising strength in conjunction with a prompt, you can create a style transfer effect.

In [ ]:
# Define the style transfer prompt
prompt = "A painting in the style of Vincent van Gogh"

# Set the denoising strength (controls the level of transformation)
denoising_strength = 0.8  # Higher value for more stylistic changes

In [ ]:
# Generate the style-transferred image
with torch.autocast(device):
    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        init_image=input_image,
        strength=denoising_strength,
    )

styled_image = output.images[0]

### 9. Display and Save the Style-Transferred Image

In [ ]:
# Save the style-transferred image
styled_image.save("output-images/styled_img2img.jpg")

In [ ]:
# Display the original and styled images
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].imshow(input_image)
ax[0].set_title("Original Image")
ax[0].axis("off")

ax[1].imshow(styled_image)
ax[1].set_title("Styled Image")
ax[1].axis("off")

plt.show()